# MCP Authentication Flow - Illustrated

This notebook illustrates the end-to-end authentication flow for Model Context Protocol (MCP). The flow involves several key steps, including metadata discovery and parsing, client registration, user authentication, token generation, and access validation.

A descripition of this flow is available in the [MCP Authentication Documentation](https://modelcontextprotocol.io/docs/tutorials/security/authorization).

This notebook illustrates the authentication flow with the GitHub MCP Server.

In [ ]:
$env:McpServer = "https://api.githubcopilot.com/mcp/"

## Step 1 - Initial Handshake

The initial handshake begins when a client application attempts to access a protected resource on the MCP server. The server responds with 401 Unauthorized status code and provides the client with the MCP metadata URL in the `WWW-Authenticate` header.


In [ ]:
curl -s -D - -X POST `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "ping"
}' `
 "$env:McpServer"

## Step 2 - Protected Resource Metadata Discovery

When receiving a 401 response in Step 1, the client extracts the MCP metadata URL from the `WWW-Authenticate` header. Then it sends a GET request to this URL to retrieve the Protected Resource Metadata (PRM) document, which contains information about the authentication server(s) that can be used to authenticate and obtain access tokens.

In [ ]:
$env:GitHub_MCP_PRM = "https://api.githubcopilot.com/.well-known/oauth-protected-resource/mcp"

curl -s -D - -X GET `
  -H "Accept: application/json" `
 "$env:GitHub_MCP_PRM"

## Step 3 - Authorization Server Discovery

The PRM document should include one or more authorization servers in the `authorization_servers` array. When there is more than one, the client can choose which one to use, but in this case there is only one available. Next, the client retrieves metadata for the authorization server. This could be either OpenID Connect (OIDC) discovery metadata or OAuth 2.0 Authorization Server Metadata, depending on what the authorization server supports. There's no clear indication in the PRM document of which type it is, so the client must try both methods.

According to the [OIDC specification], the client appends `/.well-known/openid-configuration` to the "issuer" URL to get the OIDC discovery document. The client sends a GET request to this URL and receives the OIDC metadata in response. Unfortunately, at this point all we have is the authorization server's URL, so we'll just try to use that as the issuer URL and hope for the best.

[OIDC specification]: https://openid.net/specs/openid-connect-discovery-1_0.html#ProviderConfig

In [ ]:
$env:GitHub_MCP_Auth_Server = "https://github.com/login/oauth"
$env:GitHub_MCP_OIDC_Metadata = "$env:GitHub_MCP_Auth_Server/.well-known/openid-configuration"

curl -s -X GET `
  -H "Accept: application/json" `
 "$env:GitHub_MCP_OIDC_Metadata" | jq .

What we are looking for in this metadata is the authorization endpoint, token endpoint, and supported scopes, which will be used in the subsequent steps of the authentication flow. And the scopes are clearly indicated in the `scopes_supported` field of the OIDC metadata.

But we don't see the authorization endpoint or token endpoint in the OIDC metadata. In this case, the best approach, and the one that happens to work with GitHub, is to assume the standard OAuth 2.0 endpoint paths:
- Authorization Endpoint: `{authorization_server_url}/authorize`
- Token Endpoint: `{authorization_server_url}/access_token`

In [ ]:
$env:Scopes = "openid"
$env:AuthorizeEndpoint = "$env:GitHub_MCP_Auth_Server/authorize"
$env:TokenEndpoint = "$env:GitHub_MCP_Auth_Server/access_token"

## Step 4 - Client Registration

When the authorization server supports dynamic client registration, the authorization server metadata will include a `registration_endpoint` field. The client can use this endpoint to register itself with the authorization server by sending a POST request with its details, such as redirect URIs and client name. The server responds with a client ID and client secret, which the client will use in subsequent authentication requests.

But GitHub does not support dynamic client registration, so instead we need to register our client application manually in the GitHub Developer Settings to obtain a client ID and client secret.

### Manual Client Registration

Here are the steps to manually register a client application with GitHub:

- Go to your GitHub account settings.
- Navigate to Settings > Developer settings > OAuth Apps.
- Click New OAuth App.
- Fill in the required details (application name, homepage URL, callback URL).
- After registering, GitHub will provide a client_id and a client_secret.

If your application doesn’t have a homepage, you can use a placeholder URL (such as https://example.com) for the “Homepage URL” field when registering your GitHub OAuth App. This field is required, but it does not affect the OAuth flow.

For the authorization callback URL, set the callback URI to a static page (e.g., https://localhost/callback) and, after authorization, GitHub will redirect you there with the code in the URL. You can manually copy the code from the browser’s address bar and use it to exchange for a token.

In [ ]:
$env:CallbackUrl = "https://localhost/callback"
$env:CallbackUrlEncoded = "https%3A%2F%2Flocalhost%2Fcallback"


Store the client ID and client secret securely, as they will be needed in the next steps of the authentication flow. For this illustration, the client ID and client secret are stored in a .env file which is not checked into source control.


In [ ]:

# Read .env file into the .dotenv hashtable

$dotenv = @{}

# Read the .env file line by line
Get-Content .env | ForEach-Object {
    # Match lines in KEY=VALUE format, ignoring comments and empty lines
    if ($_ -match '^(?!#)(?<key>[^=]+)=(?<value>.+)$') {
        $key = $matches['key'].Trim()
        $value = $matches['value'].Trim()
        $dotenv[$key] = $value
    }
}

## Step 5 - User Authorization

In this step, the client issues a GET request to the authorization endpoint obtained in Step 3. This request includes parameters such as the client ID, redirect URI, requested scopes, and response type (typically "code" for the authorization code flow). The user is then redirected to the authorization server's login page to authenticate and authorize the client application.

In [ ]:
curl -s -D - -X GET `
  -H "Accept: application/json" `
 "$env:AuthorizeEndpoint\?response_type=code&client_id=$($dotenv['ClientId'])&redirect_uri=$env:CallbackUrlEncoded&scope=$env:Scopes"

The response above is a HTTP 302 redirect to your specified callback URL with an authorization code included as a query parameter. The client will receive an authorization code in the query parameter at the redirect URI.

Click on the link in the "Location" header and the redirect will open in a browser. Copy the code from the URL for the next step in the authentication flow.

In [ ]:
$env:authCode="f9dca0d11d4829ff57da"

Once the client has received the authorization code, it can exchange it for an access token using the token endpoint obtained in Step 3.
The client sends a POST request to the token endpoint with parameters such as the client ID, client secret, authorization code, redirect URI, and grant type (typically "authorization_code"). The server responds with an access token (and optionally a refresh token) that the client can use to access protected resources on behalf of the user.

The response of this request will include an access_token, which you can use to authenticate subsequent requests to the GitHub API.

In [ ]:
$body = @{
    client_id = $dotenv['ClientId']
    client_secret = $dotenv['ClientSecret']
    code = $env:authCode
    redirect_uri = $env:CallbackUrl
} | ConvertTo-Json

$response = curl -s -X POST `
  -H "Content-type: application/json" `
  -H "Accept: application/json" `
  -d "$body" `
  "$env:TokenEndpoint"

$env:accessToken = ($response | ConvertFrom-Json).access_token

## Step 6 - Making Authenticated Requests

Here's an example of how to use the access token to make an authenticated request:

In [ ]:
curl -s -X POST `
  -H "Authorization: Bearer $env:accessToken" `
  -H "Content-Type: application/json" `
  -H "Accept: application/json, text/event-stream" `
  -d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "clientInfo": {
            "name": "Polyglot Notebook",
            "version": "0.1.0"
        },
        "capabilities": {},
        "protocolVersion": "2025-06-18"
    }
}' `
 "$env:McpServer" | jq .